In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload
%autoreload 2

https://medium.com/pytorch/working-on-natural-language-processing-nlp-with-pytorch-8090c879aadc

https://github.com/L1aoXingyu/pytorch-beginner/blob/master/06-Natural%20Language%20Process/N-Gram.py

In [2]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

In [3]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [4]:
trigram = [((test_sentence[i], test_sentence[i + 1]), test_sentence[i + 2])
           for i in range(len(test_sentence) - 2)]

vocb = set(test_sentence)
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [6]:
class NgramModel(nn.Module):
    def __init__(self, vocb_size, context_size, n_dim):
        super(NgramModel, self).__init__()
        self.n_word = vocb_size
        self.embedding = nn.Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size * n_dim, 128)
        self.linear2 = nn.Linear(128, self.n_word)

    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.view(1, -1)
        out = self.linear1(emb)
        out = F.relu(out)
        out = self.linear2(out)
        log_prob = F.log_softmax(out)
        return log_prob


ngrammodel = NgramModel(len(word_to_idx), CONTEXT_SIZE, 100)
criterion = nn.NLLLoss()
optimizer = optim.SGD(ngrammodel.parameters(), lr=1e-3)

In [13]:
for epoch in range(100):
    print('epoch: {}'.format(epoch + 1))
    print('*' * 10)
    running_loss = 0
    for data in trigram:
        word, label = data
        word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
        label = Variable(torch.LongTensor([word_to_idx[label]]))
        # forward
        out = ngrammodel(word)
        loss = criterion(out, label)        
        running_loss += loss.data.numpy()
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss: {:.6f}'.format(running_loss / len(word_to_idx)))

epoch: 1
**********
Loss: 5.346749
epoch: 2
**********
Loss: 5.289707
epoch: 3
**********


/tmp/ipykernel_430086/1287663233.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_prob = F.log_softmax(out)


Loss: 5.233315
epoch: 4
**********
Loss: 5.177454
epoch: 5
**********
Loss: 5.121993
epoch: 6
**********
Loss: 5.066857
epoch: 7
**********
Loss: 5.011998
epoch: 8
**********
Loss: 4.957275
epoch: 9
**********
Loss: 4.902688
epoch: 10
**********
Loss: 4.848220
epoch: 11
**********
Loss: 4.793885
epoch: 12
**********
Loss: 4.739589
epoch: 13
**********
Loss: 4.685074
epoch: 14
**********
Loss: 4.630446
epoch: 15
**********
Loss: 4.575422
epoch: 16
**********
Loss: 4.520222
epoch: 17
**********
Loss: 4.464723
epoch: 18
**********
Loss: 4.408773
epoch: 19
**********
Loss: 4.352212
epoch: 20
**********
Loss: 4.295164
epoch: 21
**********
Loss: 4.237778
epoch: 22
**********
Loss: 4.179761
epoch: 23
**********
Loss: 4.121441
epoch: 24
**********
Loss: 4.062483
epoch: 25
**********
Loss: 4.002917
epoch: 26
**********
Loss: 3.942937
epoch: 27
**********
Loss: 3.882184
epoch: 28
**********
Loss: 3.820716
epoch: 29
**********
Loss: 3.758732
epoch: 30
**********
Loss: 3.696268
epoch: 31
*********

In [23]:
word, label = trigram[5]
word = Variable(torch.LongTensor([word_to_idx[i] for i in word]))
out = ngrammodel(word)
_, predict_label = torch.max(out, 1)

/tmp/ipykernel_430086/1287663233.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_prob = F.log_softmax(out)


In [24]:
predict_word = idx_to_word[predict_label.numpy()[0]]
print('real word is {}, predict word is {}'.format(label, predict_word))

real word is And, predict word is And
